In [62]:
from pymatgen import Molecule
from pymatgen.analysis.graphs import MoleculeGraph

from moffragmentor.sbu import SBU
from pymatgen.analysis.local_env import JmolNN, CrystalNN, CutOffDictNN, VoronoiNN

VestaCutoffDictNN = CutOffDictNN.from_preset("vesta_2019")
import datetime

In [58]:
from scipy.spatial.distance import pdist
import numpy as np
def get_max_sep(coordinates):
    distances = pdist(coordinates)
    return     np.max(distances)

In [2]:
mol = Molecule.from_file('AKEDIF_clean_linker_6.xyz')

In [14]:
get_max_sep(mol.cart_coords)

7.317397078196591

Let's check if the properties get copied over

In [17]:
mol[0].properties = {'bounded': True}

In [34]:
s = mol.get_boxed_structure(10, 10, 10, reorder=False)

In [35]:
for site in s:
    print(site.properties)

{'bounded': True}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}
{'bounded': None}


In [36]:
s[0]

PeriodicSite: C (5.1557, 3.6751, 2.4358) [0.5156, 0.3675, 0.2436]

In [43]:
mg = MoleculeGraph.with_local_env_strategy(mol, VestaCutoffDictNN)

In [50]:
mol.get_distance(0,1)

1.5021651041080641

In [59]:
sbu = SBU(mol, mg, connection_indices=[1])

In [72]:
def write_tobacco_cif(sbu: SBU):
    
    max_size = get_max_sep(sbu.molecule.cart_coords)
    s = sbu.molecule.get_boxed_structure(max_size + 0.1*max_size, max_size + 0.1*max_size, max_size + 0.1*max_size, reorder=False)
    
    header_lines = [
        "data_",
        "_audit_creation_date              "
        + datetime.datetime.today().strftime("%Y-%m-%d"),
        "_audit_creation_method            'moffragmentor'",
        "_symmetry_space_group_name_H-M    'P1'",
        "_symmetry_Int_Tables_number       1",
        "_symmetry_cell_setting            triclinic",
        "loop_" "_symmetry_equiv_pos_as_xyz",
        "  x,y,z",
    ]

    cell_lines = [
        "_cell_length_a                    " + str(s.lattice.a),
        "_cell_length_b                    " + str(s.lattice.b),
        "_cell_length_c                    " + str(s.lattice.c),
        "_cell_angle_alpha                 " + str(s.lattice.alpha),
        "_cell_angle_beta                  " + str(s.lattice.beta),
        "_cell_angle_gamma                 " + str(s.lattice.gamma),
    ]

    loop_header = [
        "loop_",
        "_atom_site_label",
        "_atom_site_type_symbol",
        "_atom_site_fract_x",
        "_atom_site_fract_y",
        "_atom_site_fract_z",
        "_atom_site_charge",
    ]

    loop_content = []
    site_index = {}
    for i, site in enumerate(s):
        vec = site.frac_coords
        es = str(site.specie)

        
        if i in sbu.connection_indices:
            ind = 'X' + str(i)
            loop_content.append(
                    "{:7} {:>4} {:>15.6f} {:>15.6f} {:>15.6f} {:>15.6f}".format(
                        ind, es, vec[0], vec[1], vec[2], 0
                    )
                )
        else:
            ind = es + str(i)
            loop_content.append(
                "{:7} {:>4} {:>15.6f} {:>15.6f} {:>15.6f} {:>15.6f}".format(
                    ind, es, vec[0], vec[1], vec[2], 0
                )
            )
        site_index[i] = ind
        
    
    connection_loop_header = ["loop_", "_geom_bond_atom_site_label_1",
                              "_geom_bond_atom_site_label_2", "_geom_bond_distance", 
                              '_geom_bond_site_symmetry_1', "_ccdc_geom_bond_type"]
    
    
    connection_loop_content = []
    set_bond = set()
    for i, site in enumerate(sbu.molecule):
        neighbors = sbu.molecule_graph.get_connected_sites(0)
        for j, neighbor_site in enumerate(neighbors):
            ind0 = site_index[i]
            ind1 = site_index[j]
            
            tuple_a = (ind0, ind1)
            tuple_b = (ind1, ind0)
            
            if (not (tuple_a in set_bond) and not (tuple_b in set_bond)):
                set_bond.add(tuple_a)
                
                dist = np.round(sbu.molecule.get_distance(i,j), 3)
                # bond_type = data["bond_type"]

                connection_loop_content.append("{:7} {:>7} {:>7} {:>3}".format(ind0, ind1, dist, "S"))

    return "\n".join(header_lines+cell_lines+loop_header+loop_content+connection_loop_header+connection_loop_content)

In [73]:
write_tobacco_cif(sbu)

"data_\n_audit_creation_date              2021-01-28\n_audit_creation_method            'moffragmentor'\n_symmetry_space_group_name_H-M    'P1'\n_symmetry_Int_Tables_number       1\n_symmetry_cell_setting            triclinic\nloop__symmetry_equiv_pos_as_xyz\n  x,y,z\n_cell_length_a                    8.04913678601625\n_cell_length_b                    8.04913678601625\n_cell_length_c                    8.04913678601625\n_cell_angle_alpha                 90.0\n_cell_angle_beta                  90.0\n_cell_angle_gamma                 90.0\nloop_\n_atom_site_label\n_atom_site_type_symbol\n_atom_site_fract_x\n_atom_site_fract_y\n_atom_site_fract_z\n_atom_site_charge\nC0         C        0.519348        0.335394        0.181431        0.000000\nX1         C        0.506924        0.423602        0.345424        0.000000\nC2         C        0.576497        0.578898        0.367786        0.000000\nC3         C        0.429897        0.344090        0.478357        0.000000\nC4         C   